In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tsp-dataset/ftv170.atsp
/kaggle/input/tsp-dataset/kro124p.atsp
/kaggle/input/tsp-dataset/ftv55.atsp
/kaggle/input/tsp-dataset/rbg323.atsp
/kaggle/input/tsp-dataset/ftv33.atsp
/kaggle/input/tsp-dataset/rbg358.atsp
/kaggle/input/tsp-dataset/ftv70.atsp
/kaggle/input/tsp-dataset/rbg403.atsp
/kaggle/input/tsp-dataset/ftv38.atsp
/kaggle/input/tsp-dataset/ftv64.atsp
/kaggle/input/tsp-dataset/p43.atsp
/kaggle/input/tsp-dataset/br17.atsp
/kaggle/input/tsp-dataset/ftv47.atsp
/kaggle/input/tsp-dataset/rbg443.atsp
/kaggle/input/tsp-dataset/ry48p.atsp
/kaggle/input/tsp-dataset/ft70.atsp
/kaggle/input/tsp-dataset/ftv35.atsp
/kaggle/input/tsp-dataset/ftv44.atsp
/kaggle/input/tsp-dataset/ft53.atsp


In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import os

# Установим параметры вывода для большей читаемости
torch.set_printoptions(sci_mode=False, precision=4)
np.set_printoptions(suppress=True, precision=4)

# Словарь с оптимальными значениями для каждого файла
OPTIMAL_ROUTES = {
    'br17.atsp': 39,
    'ft53.atsp': 6905,
    'ft70.atsp': 38673,
    'ftv33.atsp': 1286,
    'ftv35.atsp': 1473,
    'ftv38.atsp': 1530,
    'ftv44.atsp': 1613,
    'ftv47.atsp': 1776,
    'ftv55.atsp': 1608,
    'ftv64.atsp': 1839,
    'ftv70.atsp': 1950,
    'kro124p.atsp': 36230,
    'p43.atsp': 5620,
    'rbg323.atsp': 1326,
    'rbg358.atsp': 1163,
    'rbg403.atsp': 2465,
    'rbg443.atsp': 2720,
    'ry48p.atsp': 14422
}

def read_atsp_file(filename):
    """Чтение файла формата ATSP и преобразование в матрицу весов"""
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f.readlines()]

    dimension = None
    matrix_data = []
    in_section = False
    
    for line in lines:
        if line.startswith('DIMENSION'):
            dimension = int(line.split(':')[1])
        elif line.startswith('EDGE_WEIGHT_SECTION'):
            in_section = True
            continue
        elif line.startswith('EOF'):
            break
        
        if in_section:
            matrix_data.extend(map(int, line.split()))
    
    return np.array(matrix_data[:dimension**2]).reshape((dimension, dimension))

def is_route_valid(route):
    """Проверка валидности маршрута (все вершины уникальны и образуют цикл)"""
    if len(route) < 2:
        return False
    
    # Проверка цикличности и уникальности вершин
    is_cycle = route[0] == route[-1]
    unique = set(route[:-1] if is_cycle else route)
    return len(unique) == (len(route)-1 if is_cycle else len(route))

def greedy_path(prob_matrix, start=0):
    """Жадное построение пути с проверкой валидности"""
    n = prob_matrix.shape[0]
    path = [start]
    visited = set(path)
    
    while len(path) < n:
        current = path[-1]
        
        # Создаем маску для непосещенных вершин
        mask = torch.ones(n, dtype=torch.bool)
        mask[list(visited)] = False
        
        # Выбираем следующую вершину только из непосещенных
        if not mask.any():
            break  # Все вершины посещены
            
        next_node = torch.argmax(prob_matrix[current][mask]).item()
        real_next_node = torch.arange(n)[mask][next_node].item()
        
        path.append(real_next_node)
        visited.add(real_next_node)
    
    # Проверяем валидность пути перед возвратом
    return path if is_route_valid(path) else None

def probabilistic_path(prob_matrix, start=0, threshold=0.0003):
    """Стохастическое построение пути с учётом порога и выбором максимума при деградации"""
    n = prob_matrix.shape[0]
    path = [start]
    
    for _ in range(n-1):
        current = path[-1]
        probs = prob_matrix[current].copy()
        
        # Обнуляем посещённые вершины
        probs[path] = 0
        
        # Применяем порог к вероятностям
        probs[probs < threshold] = 0
        
        if np.sum(probs) <= 1e-10:
            available = list(set(range(n)) - set(path))
            if not available:
                break
                
            # Выбираем вершину с максимальной исходной вероятностью
            original_probs = prob_matrix[current]
            available_indices = np.array(available)
            
            # Находим максимальную вероятность среди доступных
            max_prob = original_probs[available_indices].max()
            candidates = available_indices[original_probs[available_indices] == max_prob]
            
            next_node = np.random.choice(candidates)
        else:
            probs /= probs.sum()
            next_node = np.random.choice(n, p=probs)
        
        path.append(next_node)
    
    return path if is_route_valid(path) else None
    
def calculate_route_length(path, distance_matrix):
    """Безопасный расчет длины маршрута"""
    if path is None or not is_route_valid(path):
        return float('inf'), []
    
    # Проверяем длину пути
    if len(path) == distance_matrix.shape[0] and path[0] != path[-1]:
        # Замыкаем цикл только если нужно
        path.append(path[0])
    elif len(path) - 1 != distance_matrix.shape[0] or path[0] != path[-1]:
        return float('inf'), []
       
    total = 0
    for i in range(len(path)-1):
        total += distance_matrix[path[i], path[i+1]]
    return total, path

def train_model(distance_matrix, penalty=3.0, epochs=1000, lr=0.05):
    """Функция обучения модели"""
    # Нормализация с защитой от деления на ноль
    values = distance_matrix[~torch.eye(distance_matrix.size(0), dtype=torch.bool)]
    min_val = values.min()
    max_val = values.max()
    
    if (max_val - min_val) < 1e-6:
        S_norm = torch.zeros_like(distance_matrix)
    else:
        S_norm = (distance_matrix - min_val) / (max_val - min_val)
    
    S_norm.fill_diagonal_(0.0)    

    # Инициализация весов
    weights = torch.zeros(S_norm.shape[0], S_norm.shape[1], requires_grad=True)
    
    optimizer = torch.optim.Adam([weights], lr=lr)
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        
        masked = weights.clone().fill_diagonal_(-np.inf)
        soft_weights = F.softmax(masked, dim=1)
        
        main_loss = torch.sum(soft_weights * S_norm)
        penalty_loss = torch.sum((soft_weights.sum(0) - 1) ** 2)
        loss = main_loss + penalty * penalty_loss
        
        loss.backward()
        optimizer.step()
        
    with torch.no_grad():
        masked = weights.clone().fill_diagonal_(-np.inf)
        return F.softmax(masked, dim=1).numpy()

def combine_graphs(graphs, optimal_lengths):
    """
    Объединение графов в один большой граф с соединительными рёбрами.
    Игнорирует диагональные элементы при расчете максимального ребра.
    
    :param graphs: Список матриц смежности графов
    :param optimal_lengths: Список оптимальных длин для каждого графа
    :return: Объединённая матрица смежности и общая оптимальная длина
    """
    if len(graphs) == 0:
        print("Список матриц смежности графов пуст")
        return [], 0
        
    # Находим максимальное недиагональное ребро среди всех графов
    max_non_diag = 0
    for g in graphs:
        # Создаем маску для недиагональных элементов
        mask = ~np.eye(g.shape[0], dtype=bool)
        non_diag_elements = g[mask]
        
        if non_diag_elements.size > 0:
            current_max = non_diag_elements.max()
            if current_max > max_non_diag:
                max_non_diag = current_max
    
    # Если все графы состоят только из диагонали, устанавливаем вес по умолчанию
    if max_non_diag == 0:
        max_non_diag = 100  # Значение по умолчанию
    
    connection_weight = 2 * max_non_diag

    # Вычисляем размеры каждого графа
    sizes = [g.shape[0] for g in graphs]
    total_size = sum(sizes)
    
    # Создаём пустую матрицу для объединённого графа
    combined = np.full((total_size, total_size), connection_weight, dtype=np.float32)
    
    # Заполняем диагональные блоки исходными графами (с сохранением их диагонали)
    start_idx = 0
    for i, g in enumerate(graphs):
        end_idx = start_idx + sizes[i]
        combined[start_idx:end_idx, start_idx:end_idx] = g
        start_idx = end_idx
    
    # Вычисляем общую оптимальную длину
    num_graphs = len(graphs)
    if num_graphs == 1:
        total_optimal = sum(optimal_lengths)
    elif num_graphs > 1:
        total_optimal = sum(optimal_lengths) + connection_weight * num_graphs
    
    return combined, total_optimal

# Основной блок выполнения
if __name__ == "__main__":
    # Параметры алгоритма
    NUM_RUNS = 100
    START_NODE = 0
    BASE_DIR = '/kaggle/input/tsp-dataset/'
    
    results = []
    all_graphs = []
    actual_optimal = []

    for filename, optimal in OPTIMAL_ROUTES.items():
        full_path = os.path.join(BASE_DIR, filename)
        if not os.path.exists(full_path):
            print(f"Файл {filename} не найден, пропускаем.")
            continue
            
        print(f"\n{'='*50}")
        print(f"Обработка файла: {filename}")
        
        # Загрузка данных
        dist_matrix = read_atsp_file(full_path)
        all_graphs.append(dist_matrix)
        actual_optimal.append(optimal)
        S_tensor = torch.tensor(dist_matrix, dtype=torch.float32)
        
        # Обучение модели
        transition_probs = train_model(S_tensor)
        print("Обучение модели завершено")
        
        # Жадный поиск
        greedy_route = greedy_path(torch.tensor(transition_probs), START_NODE)
        greedy_length, _ = calculate_route_length(greedy_route, dist_matrix)
        
        # Случайный поиск
        best_random_length = float('inf')
        for _ in range(NUM_RUNS):
            route = probabilistic_path(transition_probs, START_NODE)
            length, _ = calculate_route_length(route, dist_matrix)
            if length < best_random_length:
                best_random_length = length
        
        # Расчет отклонений
        greedy_dev = (greedy_length - optimal)/optimal * 100 if greedy_length != float('inf') else float('inf')
        random_dev = (best_random_length - optimal)/optimal * 100 if best_random_length != float('inf') else float('inf')
        
        results.append({
            'filename': filename,
            'optimal': optimal,
            'greedy': greedy_length,
            'random': best_random_length,
            'greedy_dev': greedy_dev,
            'random_dev': random_dev
        })
        
        print(f"Оптимальная длина: {optimal}")
        print(f"Жадный алгоритм: {greedy_length} ({greedy_dev:.1f}%)")
        print(f"Случайный поиск: {best_random_length} ({random_dev:.1f}%)")

    # Обработка объединённого графа
    if all_graphs:
        print("\n\nОбъединение графов...")
        combined_graph, combined_optimal = combine_graphs(all_graphs, actual_optimal)
    
        # Вывод информации
        print(f"\nОбщий граф создан:")
        print(f"  - Количество вершин: {combined_graph.shape[0]}")
        print(f"  - Количество рёбер: {combined_graph.size}")
        print(f"  - Оптимальная длина маршрута: {combined_optimal}")
        
        print(f"\n{'='*50}")
        print(f"Обработка объединённого графа")
        
        S_tensor = torch.tensor(combined_graph, dtype=torch.float32)
        
        # Обучение модели
        transition_probs = train_model(S_tensor)
        print("Обучение модели завершено")
        
        # Жадный поиск
        greedy_route = greedy_path(torch.tensor(transition_probs), START_NODE)
        greedy_length, _ = calculate_route_length(greedy_route, combined_graph)
        
        # Случайный поиск
        best_random_length = float('inf')
        for _ in range(NUM_RUNS):
            route = probabilistic_path(transition_probs, START_NODE)
            length, _ = calculate_route_length(route, combined_graph)
            if length < best_random_length:
                best_random_length = length
        
        # Расчет отклонений
        greedy_dev = (greedy_length - combined_optimal)/combined_optimal * 100 if greedy_length != float('inf') else float('inf')
        random_dev = (best_random_length - combined_optimal)/combined_optimal * 100 if best_random_length != float('inf') else float('inf')
        
        results.append({
            'filename': f'combined{combined_graph.shape[0]}',
            'optimal': combined_optimal,
            'greedy': greedy_length,
            'random': best_random_length,
            'greedy_dev': greedy_dev,
            'random_dev': random_dev
        })
        
        print(f"Оптимальная длина: {combined_optimal}")
        print(f"Жадный алгоритм: {greedy_length} ({greedy_dev:.1f}%)")
        print(f"Случайный поиск: {best_random_length} ({random_dev:.1f}%)")

    # Вывод сводной таблицы
    print("\n\nРезультаты:")
    print(f"{'Файл':<15} | {'Оптимум':<10} | {'Жадный':<10} | {'Случ.':<10} | {'Откл. жадный (%)':<15} | {'Откл. случ. (%)':<15}")
    print("-"*85)
    for res in results:
        # Форматирование отклонений
        greedy_dev_str = (
            f"{res['greedy_dev']:.1f}%" 
            if isinstance(res['greedy_dev'], float) 
            else "INF"
        )
        random_dev_str = (
            f"{res['random_dev']:.1f}%" 
            if isinstance(res['random_dev'], float) 
            else "INF"
        )
        
        # Форматирование длин маршрутов
        greedy_length = (
            res['greedy'] 
            if res['greedy'] != float('inf') 
            else "INF"
        )
        random_length = (
            res['random'] 
            if res['random'] != float('inf') 
            else "INF"
        )
        
        print(
            f"{res['filename']:<15} | "
            f"{res['optimal']:<10} | "
            f"{greedy_length:<10} | "
            f"{random_length:<10} | "
            f"{greedy_dev_str:<15} | "
            f"{random_dev_str:<15}"
        )


Обработка файла: br17.atsp
Обучение модели завершено
Оптимальная длина: 39
Жадный алгоритм: 39 (0.0%)
Случайный поиск: 39 (0.0%)

Обработка файла: ft53.atsp
Обучение модели завершено
Оптимальная длина: 6905
Жадный алгоритм: 7967 (15.4%)
Случайный поиск: 7543 (9.2%)

Обработка файла: ft70.atsp
Обучение модели завершено
Оптимальная длина: 38673
Жадный алгоритм: 40410 (4.5%)
Случайный поиск: 39455 (2.0%)

Обработка файла: ftv33.atsp
Обучение модели завершено
Оптимальная длина: 1286
Жадный алгоритм: 1616 (25.7%)
Случайный поиск: 1474 (14.6%)

Обработка файла: ftv35.atsp
Обучение модели завершено
Оптимальная длина: 1473
Жадный алгоритм: 1893 (28.5%)
Случайный поиск: 1705 (15.8%)

Обработка файла: ftv38.atsp
Обучение модели завершено
Оптимальная длина: 1530
Жадный алгоритм: 1942 (26.9%)
Случайный поиск: 1763 (15.2%)

Обработка файла: ftv44.atsp
Обучение модели завершено
Оптимальная длина: 1613
Жадный алгоритм: 1990 (23.4%)
Случайный поиск: 1964 (21.8%)

Обработка файла: ftv47.atsp
Обучение 